In [1]:
from etr_case_generator.generator import ETRCaseGenerator
from etr_case_generator.reasoning_problem import ReasoningProblem
from etr_case_generator.ontology import ELEMENTS

from pyetr import View, State, SetOfStates
from pyetr.inference import default_inference_procedure, default_procedure_does_it_follow

In [2]:
r = ReasoningProblem(generator=ETRCaseGenerator(ELEMENTS))
r.update_premises(premises=[View.from_str("Ax {C(x)}")])

In [3]:
r.update_query(query=View.from_str("{C(B())}"))

In [4]:
default_procedure_does_it_follow([View.from_str("Ax {C(x)}")], View.from_str("{C(B())}"))

False

In [5]:
print(r.full_prose())

Consider the following premises:

1. For all X, X is quantum-stable.

Does it follow that stellarite is quantum-stable?

Answer using 'YES' or 'NO' ONLY.


In [6]:
r.etr_conclusion

({0}, '')

In [7]:
view_queue = [View.get_falsum()]

In [9]:
from pyetr import View, State, SetOfStates

from etr_case_generator.mutate_view import random_existential_quantify, random_universal_quantify

v = View.from_str("{P(a())Q(b())R(c())}")
v = random_universal_quantify(v)
v = random_existential_quantify(v)
v = random_universal_quantify(v)
v

{P(y)Q(b)R(c)}
{y, b, c}
U={y} E={}
U={y} E={}
{P(u)Q(b)R(p)}
{p, u, b}
U={y,u} E={p}
U={u} E={p}


∃p ∀u {P(u)Q(b())R(p)}

In [18]:
from pyetr import DependencyRelation, ArbitraryObject

DependencyRelation(
    universals=[ArbitraryObject(name="x")],
    existentials=[],
    dependencies=[],
).fusion(
    DependencyRelation(
    universals=[],
    existentials=[ArbitraryObject(name="y")],
    dependencies=[],
))

U={x} E={y}

In [16]:
v = view_queue.pop(0)
if not (v.is_falsum or v.is_verum): view_queue.append(v)
view_queue.append(mutate_view(v))
[v for v in view_queue]

[∀x {R(x)},
 ∀x {R(x)},
 ∀x {R(x)},
 ∀x {~R(x)},
 {R(U())},
 {}^{},
 {~R(U())},
 ∀x {~R(x)},
 {A(T())R(U())},
 {A(T())}^{},
 {~A(T()),~R(U())},
 ∃x {~A(x),~R(U())}]

In [19]:
from etr_case_generator.generator import ETRCaseGenerator
from etr_case_generator.ontology import ELEMENTS

g = ETRCaseGenerator(ELEMENTS)

g.view_to_natural_language(view_queue[-1])

('there is a(n) X such that either X is not corrosive, or harmonium is not magnetic',
 {'R': 'magnetic', 'A': 'corrosive', 'U': 'harmonium'})

In [18]:
g.view_to_natural_language(g.add_quantification(view_queue[0]))

AttributeError: 'ETRCaseGenerator' object has no attribute 'add_quantification'

[{}]

In [7]:
m.merge_random_unary_predicate()

In [8]:
m.view

{S(L())B()I(B()),A()S(L())I(B())}^{C()}

In [13]:
[a.predicate.name for a in m.view.atoms]

['A', 'C', 'B']

In [4]:
m.factor_random_atom()
m.view

{B()}^{C()}

In [8]:
m.factor_random_atom()
m.view

{C()}^{}

In [ ]:
st = list(View.from_str("{A()B()}").stage)[0]
from pyetr.view import state_division
state_division(st, )

AttributeError: 'State' object has no attribute 'division'

In [106]:
ab = View.from_str("{A(), B()}")
a = View.from_str("{A()}")
b = View.from_str("{B()}")
c = View.from_str("{C()}")

In [125]:
res = View.from_str("{p()q()r(), q()s(), p()}").factor(View.from_str("{p(), q()}"))
res

{s(),0,q()r()p()}

In [129]:
res.answer(View.get_verum())

{s(),0,q()r()p()}

In [126]:
res.answer(res)

{q()r()p()}

In [115]:
ab.product(View.get_verum())

{A(),B()}

In [121]:
ans = ab.factor(a)

In [123]:
ans.answer(ans)

{B()}

In [73]:
a = list(v.atoms)[0]
a_v = View.with_defaults(stage=SetOfStates({State({a})}))

In [93]:
a_v

{A()}

In [90]:
v.factor(a_v) #.product(View.get_falsum())

{B(),0}

In [46]:
v.division(View.from_str("{A()}")).factor(View.get_verum())

{0,B()}

In [40]:
v.merge(View.from_str("{C()}"))

{C()B(),A()C()}

# Editing Views with Generator

In [2]:
g = ETRCaseGenerator(ontology=ELEMENTS)

In [3]:
view = View.from_str("{P(a()), Q(b())}^{R(a())}")

In [4]:
# ret = g.add_quantification(view)

In [12]:
a = list(view.atoms)[0]
t = a.terms[0]
t

b

In [13]:
new_t = list(view.atoms)[2].terms[0]

In [14]:
new_t

a

In [15]:
g.replace_term_in_atom(a, term_to_replace=t, replacement_term=new_t)

Q(a)